In [11]:
from google_drive_downloader import GoogleDriveDownloader as gdd

In [12]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1NkFzQ5N-I1J34-0uStvw0xaPbntqk7AV',
dest_path='content/rosseman.zip',   
unzip=True)

In [ ]:
# since the size of teras package is too big I have done Rnn in google collab.

In [2]:
from IPython.display import Markdown, display, Image
import warnings
warnings.filterwarnings('ignore')

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import numpy as np
from statsmodels.tsa.stattools import adfuller, acf, pacf
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, RobustScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
sns.set()

In [7]:
def read_csv(csv_path, missing_values=[]):
  """
  reads csv file
  """
  try:
    df = pd.read_csv(csv_path, na_values=missing_values)
    print("file read as csv")
    return df
  except FileNotFoundError:
    print("file not found")

In [35]:
df_train = read_csv("content/rossman data/train.csv")
test = read_csv("content/rossman data/test.csv")

file read as csv
file read as csv


In [36]:
# A quick view a data on hand
display(df_train.head())


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [38]:
display(df_train.describe())


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


In [40]:
display(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


None

In [41]:
def get_feature(df):
  """
  gets date and sales column
  """
  return df[["Date", "Sales"]]

In [43]:
def drop_closed(df):
  """
  drops rows when open column is zero
  """
  df = df[df['Open'] == 1]
  return df

In [42]:

def scaler(df, columns, mode="minmax"):
  """
  scales using MinMax scaler
  """
  if (mode == "minmax"):
      minmax_scaler = MinMaxScaler()
      return pd.DataFrame(minmax_scaler.fit_transform(df), columns=columns), minmax_scaler
  elif (mode == "standard"):
      scaler = StandardScaler()
      return pd.DataFrame(scaler.fit_transform(df), columns=columns), scaler
  elif (mode == "robust"):
      scaler = RobustScaler()
      return pd.DataFrame(scaler.fit_transform(df), columns=columns), scaler

In [32]:
def add_scaled_sales(df):
  """
  adds scaled sales
  """
  scaled_sales, scaler_object = scaler(df[["Sales"]], mode="minmax", columns=["scaled_sales"])
  df["scaled_sales"] = scaled_sales["scaled_sales"].to_list()
  return df, scaler_object

In [45]:
def handle_outliers(df, col, method="lower_upper"):
  """
  handles outlier in the data
  """
        
  df = df.copy()
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)

  lower_bound = q1 - ((1.5) * (q3 - q1))
  upper_bound = q3 + ((1.5) * (q3 - q1))

  if method == "mean":
    df[col] = np.where(df[col] < lower_bound,   df[col].mean(), df[col])
    df[col] = np.where(df[col] > upper_bound, df[col].mean(), df[col])

  elif  method == "mode":
    df[col] = np.where(df[col] < lower_bound,   df[col].mode()[0], df[col])
    df[col] = np.where(df[col] > upper_bound, df[col].mode()[0], df[col])
  else:
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

  return df

In [47]:
df_train["Sales"] = df_train["Sales"].astype("float")

In [46]:
df_train = drop_closed(df_train)
df_train = get_feature(df_train)
df_train = handle_outliers(df_train, "Sales")

In [49]:
df_train_agg = df_train.groupby("Date").agg({"Sales": "mean"})
df_train_agg.describe()

,Sales
count,942.000000
mean,6988.142162
std,1475.510458
min,4057.782960
25%,5725.756766
50%,6822.535825
75%,7778.867102
max,11908.661883
